<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chains" data-toc-modified-id="Chains-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Chains</a></span></li><li><span><a href="#Agents" data-toc-modified-id="Agents-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Agents</a></span></li><li><span><a href="#Memory" data-toc-modified-id="Memory-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Memory</a></span></li><li><span><a href="#Callbacks" data-toc-modified-id="Callbacks-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Callbacks</a></span></li><li><span><a href="#Put-Together" data-toc-modified-id="Put-Together-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Put Together</a></span><ul class="toc-item"><li><span><a href="#Create-Search-Assistant" data-toc-modified-id="Create-Search-Assistant-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Create Search Assistant</a></span></li></ul></li></ul></div>

In [2]:
!python --version

Python 3.10.12


In [3]:
!pip install -q langchain openai

# Chains

In [4]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

In [5]:
import os
os.environ['OPENAI_API_KEY'] = 'your api key here'

In [5]:
llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [6]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [7]:
print(chain.run("colorful socks"))



Cheery Toes Socks.


# Agents

In [8]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)

In [9]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

In [10]:
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent

system_message = SystemMessage(content="You are very powerful assistant, but bad at calculating lengths of words.")
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)

In [11]:
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

In [12]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [13]:
agent_executor.run("how many letters in the word educa?")



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


'There are 5 letters in the word "educa".'

# Memory

In [14]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)

In [15]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)

In [16]:
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

In [22]:
agent_executor.run("how many letters in the word educa?")



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


'There are 5 letters in the word "educa".'

In [23]:
agent_executor.run("is that a real word?")



> Entering new AgentExecutor chain...
No, "educa" is not a real word in English.

> Finished chain.


'No, "educa" is not a real word in English.'

# Callbacks

In [24]:
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

In [25]:
handler = StdOutCallbackHandler()
llm = OpenAI()
prompt = PromptTemplate.from_template("1 + {number} = ")

In [26]:
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])

In [27]:
chain.run(number=2)



> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


'\n\n3'

In [28]:
chain = LLMChain(llm=llm, prompt=prompt)
chain.run(number=2, callbacks=[handler])



> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


'\n3'

# Put Together

## Create Search Assistant
With Serp API & LLM-Math

In [19]:
!pip install -q langchain openai google-search-results

In [20]:
# Set env var OPENAI_API_KEY and SERPAPI_API_KEY or load from a .env file
import dotenv
dotenv.load_dotenv()

True

In [21]:
# Memory
from langchain.memory import ConversationBufferMemory
MEMORY_KEY = "chat_history"
memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)

In [24]:
# Prompt
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent
from langchain.prompts import MessagesPlaceholder

system_message = SystemMessage(content="You are a search assistant.")
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)

In [25]:
# Tool
from langchain.agents import load_tools
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [27]:
# Agent
from langchain.agents import AgentExecutor

search_agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt, memory=memory)

In [28]:
# Agent Executor
agent_executor = AgentExecutor(agent=search_agent, tools=tools, memory=memory, verbose=True)

In [29]:
# Run
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...

Invoking: `Search` with `{'query': 'population of Canada in 2023'}`


38,781,291As of 2023, the estimated population of Canada is approximately 38,781,291 people.

> Finished chain.


'As of 2023, the estimated population of Canada is approximately 38,781,291 people.'

In [32]:
agent_executor.run("how much does it differ from the population in 2022?")



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `38781291 - 37689100`


Answer: 1092191The population of Canada in 2023 is estimated to be approximately 1,092,191 higher than the population in 2022.

> Finished chain.


'The population of Canada in 2023 is estimated to be approximately 1,092,191 higher than the population in 2022.'

In [33]:
agent_executor.run("what is the square root of 121?")



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `sqrt(121)`


Answer: 11.0The square root of 121 is 11.

> Finished chain.


'The square root of 121 is 11.'